In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Dropout,LSTM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Raw_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/only data/raw_data.npz'

Theta_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time domain /RAW/Theta_time.npz'



# RAW = np.load(Raw_data, allow_pickle=True)  # Allow loading pickled object arrays
# X = RAW['X'].astype('float64')
# Y = RAW['Y'].astype('float64')
# group = RAW['Group'].astype('float64')

This is a good performing model

# CNN-LSTM

In [ ]:
%%capture
!pip install tensorflow_addons

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_callback = ModelCheckpoint(
                f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Theta/CNN_LSTM/best_model_{run_name}.h5',
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Theta/CNN_LSTM/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[

              , csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Evaluate client model
            y_pred = (client_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
global_model, metrics_df = federated_learning(Theta_data)


Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 18, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 9, 256)            0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 31s 153ms/step - loss: 0.6929 - accuracy: 0.5046 - val_loss: 0.6931 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 1s 30ms/step - loss: 0.6927 - accuracy: 0.5038 - val_loss: 0.6931 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 1s 26ms/step - loss: 0.6925 - accuracy: 0.5038 - val_loss: 0.6931 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 1s 21ms/step - loss: 0.6921 - accuracy: 0.5038 - val_loss: 0.6931 - val_accuracy: 0.4849
Epoch 5/100
29/29 [==============================] - 1s 42ms/step - loss: 0.6918 - accuracy: 0.5038 - val_loss: 0.6930 - val_accuracy: 0.4881
Epoch 6/100
29/29 [==============================] - 1s 39ms/step - loss: 0.6915 - accuracy: 0.5038 - val_loss: 0.6929 - val_accuracy: 0.6142
Epoch 7/100
29/29 [==============================] - 1s 39ms/step - loss: 0.6910 - accuracy: 0.5038 - val_loss: 0.6929 - val_accuracy: 0.5453
Epoch 8/100
29/2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 9s 79ms/step - loss: 0.6930 - accuracy: 0.4989 - val_loss: 0.6931 - val_accuracy: 0.4955
Epoch 2/100
28/28 [==============================] - 0s 15ms/step - loss: 0.6927 - accuracy: 0.5011 - val_loss: 0.6931 - val_accuracy: 0.4955
Epoch 3/100
28/28 [==============================] - 0s 16ms/step - loss: 0.6925 - accuracy: 0.5011 - val_loss: 0.6931 - val_accuracy: 0.4955
Epoch 4/100
28/28 [==============================] - 1s 23ms/step - loss: 0.6922 - accuracy: 0.5011 - val_loss: 0.6931 - val_accuracy: 0.5622
Epoch 5/100
28/28 [==============================] - 1s 23ms/step - loss: 0.6919 - accuracy: 0.5011 - val_loss: 0.6930 - val_accuracy: 0.6131
Epoch 6/100
28/28 [==============================] - 0s 13ms/step - loss: 0.6915 - accuracy: 0.5011 - val_loss: 0.6930 - val_accuracy: 0.5373
Epoch 7/100
28/28 [==============================] - 0s 15ms/step - loss: 0.6907 - accuracy: 0.5025 - val_loss: 0.6930 - val_accuracy: 0.5045
Epoch 8/100
28/28 

...

Epoch 2/100
29/29 [==============================] - 0s 13ms/step - loss: 0.3095 - accuracy: 0.8688 - val_loss: 0.6972 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 1s 23ms/step - loss: 0.2920 - accuracy: 0.8739 - val_loss: 0.6859 - val_accuracy: 0.4903
Epoch 4/100
29/29 [==============================] - 1s 21ms/step - loss: 0.2801 - accuracy: 0.8798 - val_loss: 0.6766 - val_accuracy: 0.5323
Epoch 5/100
29/29 [==============================] - 1s 22ms/step - loss: 0.2854 - accuracy: 0.8847 - val_loss: 0.6662 - val_accuracy: 0.6390
Epoch 6/100
29/29 [==============================] - 1s 24ms/step - loss: 0.2789 - accuracy: 0.8863 - val_loss: 0.6617 - val_accuracy: 0.6552
Epoch 7/100
29/29 [==============================] - 1s 22ms/step - loss: 0.2808 - accuracy: 0.8820 - val_loss: 0.6565 - val_accuracy: 0.6562
Epoch 8/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2681 - accuracy: 0.8955 - val_loss: 0.6616 - val_accuracy: 0.6034
Epoch 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 8s 77ms/step - loss: 0.3271 - accuracy: 0.8616 - val_loss: 0.7001 - val_accuracy: 0.4955
Epoch 2/100
28/28 [==============================] - 1s 32ms/step - loss: 0.2967 - accuracy: 0.8746 - val_loss: 0.6944 - val_accuracy: 0.4966
Epoch 3/100
28/28 [==============================] - 1s 33ms/step - loss: 0.2888 - accuracy: 0.8681 - val_loss: 0.6827 - val_accuracy: 0.5057
Epoch 4/100
28/28 [==============================] - 1s 27ms/step - loss: 0.2998 - accuracy: 0.8710 - val_loss: 0.6732 - val_accuracy: 0.5645
Epoch 5/100
28/28 [==============================] - 1s 24ms/step - loss: 0.2849 - accuracy: 0.8786 - val_loss: 0.6680 - val_accuracy: 0.6425
Epoch 6/100
28/28 [==============================] - 0s 15ms/step - loss: 0.2679 - accuracy: 0.8871 - val_loss: 0.6644 - val_accuracy: 0.6391
Epoch 7/100
28/28 [==============================] - 0s 15ms/step - loss: 0.2697 - accuracy: 0.8879 - val_loss: 0.6607 - val_accuracy: 0.6391
Epoch 8/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 7s 65ms/step - loss: 0.3506 - accuracy: 0.8463 - val_loss: 0.7034 - val_accuracy: 0.4855
Epoch 2/100
31/31 [==============================] - 1s 20ms/step - loss: 0.3043 - accuracy: 0.8739 - val_loss: 0.6985 - val_accuracy: 0.4855
Epoch 3/100
31/31 [==============================] - 1s 29ms/step - loss: 0.2963 - accuracy: 0.8744 - val_loss: 0.6850 - val_accuracy: 0.4917
Epoch 4/100
31/31 [==============================] - 1s 32ms/step - loss: 0.2882 - accuracy: 0.8811 - val_loss: 0.6741 - val_accuracy: 0.5568
Epoch 5/100
31/31 [==============================] - 1s 30ms/step - loss: 0.3011 - accuracy: 0.8731 - val_loss: 0.6648 - val_accuracy: 0.6488
Epoch 6/100
31/31 [==============================] - 0s 15ms/step - loss: 0.2882 - accuracy: 0.8860 - val_loss: 0.6629 - val_accuracy: 0.6322
Epoch 7/100
31/31 [==============================] - 0s 14ms/step - loss: 0.2888 - accuracy: 0.8804 - val_loss: 0.6582 - val_accuracy: 0.6281
Epoch 8/100
31/31 

In [ ]:

metrics_df.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.662,0.691,0.588,0.635,0.588,0.737,0.325
1,1,0.691,0.700,0.669,0.684,0.669,0.713,0.383
2,2,0.640,0.605,0.807,0.691,0.807,0.472,0.279
3,0,0.668,0.722,0.548,0.623,0.548,0.789,0.337
4,1,0.687,0.748,0.565,0.644,0.565,0.809,0.374
5,2,0.628,0.592,0.819,0.687,0.819,0.436,0.255
6,0,0.698,0.739,0.611,0.669,0.611,0.784,0.395
7,1,0.690,0.660,0.785,0.717,0.785,0.595,0.380
8,2,0.701,0.712,0.675,0.693,0.675,0.727,0.402
9,0,0.713,0.694,0.762,0.726,0.762,0.663,0.425


In [ ]:
metrics_df.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/Time Domain/CNN_LSTM/Theta_time_CNN_LSTM.csv', index = False)

# CNN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(LSTM(256, return_sequences=True))
    # model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Theta/CNN/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Theta/CNN/training_log_{run_name}.csv', append=True, separator=';')


            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)


            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


In [ ]:
global_model_CNN, metrics_df_CNN = federated_learning(Theta_data)

Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 18, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 9, 256)            0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 5s 44ms/step - loss: 1.7720 - accuracy: 0.5024 - val_loss: 1.7672 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 0s 14ms/step - loss: 1.7568 - accuracy: 0.5436 - val_loss: 1.7579 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 0s 13ms/step - loss: 1.7410 - accuracy: 0.6013 - val_loss: 1.7486 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 1s 19ms/step - loss: 1.7272 - accuracy: 0.5975 - val_loss: 1.7390 - val_accuracy: 0.4881
Epoch 5/100
29/29 [==============================] - 0s 16ms/step - loss: 1.7128 - accuracy: 0.6026 - val_loss: 1.7295 - val_accuracy: 0.5517
Epoch 6/100
29/29 [==============================] - 0s 17ms/step - loss: 1.6997 - accuracy: 0.6180 - val_loss: 1.7196 - val_accuracy: 0.6584
Epoch 7/100
29/29 [==============================] - 0s 10ms/step - loss: 1.6836 - accuracy: 0.6363 - val_loss: 1.7094 - val_accuracy: 0.6552
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 0s 12ms/step - loss: 1.7581 - accuracy: 0.5512 - val_loss: 1.7582 - val_accuracy: 0.4955
Epoch 3/100
28/28 [==============================] - 0s 12ms/step - loss: 1.7445 - accuracy: 0.5722 - val_loss: 1.7492 - val_accuracy: 0.4966
Epoch 4/100
28/28 [==============================] - 0s 16ms/step - loss: 1.7318 - accuracy: 0.5976 - val_loss: 1.7401 - val_accuracy: 0.5950
Epoch 5/100
28/28 [==============================] - 0s 17ms/step - loss: 1.7179 - accuracy: 0.6163 - val_loss: 1.7309 - val_accuracy: 0.6222
Epoch 6/100
28/28 [==============================] - 1s 24ms/step - loss: 1.7029 - accuracy: 0.6152 - val_loss: 1.7214 - val_accuracy: 0.6527
Epoch 7/100
28/28 [==============================] - 0s 10ms/step - loss: 1.6876 - accuracy: 0.6353 - val_loss: 1.7128 - val_accuracy: 0.5385
Epoch 8/100
28/28 [==============================] - 0s 13ms/step - loss: 1.6745 - accuracy: 0.6276 - val_loss: 1.7024 - val_accuracy: 0.6018
Epoch 9/100
28/28 

...

29/29 [==============================] - 0s 10ms/step - loss: 0.4253 - accuracy: 0.9410 - val_loss: 0.9957 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 0s 11ms/step - loss: 0.4203 - accuracy: 0.9459 - val_loss: 0.9895 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 0s 16ms/step - loss: 0.4225 - accuracy: 0.9432 - val_loss: 0.9866 - val_accuracy: 0.4871
Epoch 5/100
29/29 [==============================] - 1s 18ms/step - loss: 0.4177 - accuracy: 0.9448 - val_loss: 0.9600 - val_accuracy: 0.5183
Epoch 6/100
29/29 [==============================] - 1s 21ms/step - loss: 0.4123 - accuracy: 0.9472 - val_loss: 0.9467 - val_accuracy: 0.5312
Epoch 7/100
29/29 [==============================] - 1s 23ms/step - loss: 0.4055 - accuracy: 0.9520 - val_loss: 0.9269 - val_accuracy: 0.5711
Epoch 8/100
29/29 [==============================] - 0s 14ms/step - loss: 0.4040 - accuracy: 0.9529 - val_loss: 0.9262 - val_accuracy: 0.5690
Epoch 9/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 0s 17ms/step - loss: 0.4667 - accuracy: 0.9143 - val_loss: 0.9866 - val_accuracy: 0.4955
Epoch 3/100
28/28 [==============================] - 0s 16ms/step - loss: 0.4238 - accuracy: 0.9417 - val_loss: 0.9832 - val_accuracy: 0.4955
Epoch 4/100
28/28 [==============================] - 1s 22ms/step - loss: 0.4177 - accuracy: 0.9471 - val_loss: 0.9795 - val_accuracy: 0.4977
Epoch 5/100
28/28 [==============================] - 1s 26ms/step - loss: 0.4209 - accuracy: 0.9392 - val_loss: 0.9692 - val_accuracy: 0.5057
Epoch 6/100
28/28 [==============================] - 1s 23ms/step - loss: 0.4083 - accuracy: 0.9522 - val_loss: 0.9476 - val_accuracy: 0.5238
Epoch 7/100
28/28 [==============================] - 1s 18ms/step - loss: 0.4067 - accuracy: 0.9493 - val_loss: 0.9407 - val_accuracy: 0.5396
Epoch 8/100
28/28 [==============================] - 0s 17ms/step - loss: 0.4019 - accuracy: 0.9539 - val_loss: 0.9237 - val_accuracy: 0.5769
Epoch 9/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 1s 16ms/step - loss: 0.4583 - accuracy: 0.9181 - val_loss: 0.9929 - val_accuracy: 0.4855
Epoch 3/100
31/31 [==============================] - 0s 15ms/step - loss: 0.4378 - accuracy: 0.9349 - val_loss: 0.9894 - val_accuracy: 0.4855
Epoch 4/100
31/31 [==============================] - 1s 22ms/step - loss: 0.4521 - accuracy: 0.9230 - val_loss: 0.9777 - val_accuracy: 0.4938
Epoch 5/100
31/31 [==============================] - 1s 20ms/step - loss: 0.4352 - accuracy: 0.9346 - val_loss: 0.9602 - val_accuracy: 0.5145
Epoch 6/100
31/31 [==============================] - 1s 21ms/step - loss: 0.4313 - accuracy: 0.9336 - val_loss: 0.9452 - val_accuracy: 0.5465
Epoch 7/100
31/31 [==============================] - 1s 23ms/step - loss: 0.4233 - accuracy: 0.9390 - val_loss: 0.9120 - val_accuracy: 0.6043
Epoch 8/100
31/31 [==============================] - 1s 22ms/step - loss: 0.4212 - accuracy: 0.9429 - val_loss: 0.9091 - val_accuracy: 0.6147
Epoch 9/100
31/31 

In [ ]:
metrics_df_CNN.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.657,0.698,0.554,0.618,0.554,0.760,0.315
1,1,0.682,0.722,0.593,0.651,0.593,0.771,0.364
2,2,0.662,0.676,0.620,0.647,0.620,0.703,0.323
3,0,0.681,0.739,0.559,0.637,0.559,0.802,0.362
4,1,0.708,0.725,0.672,0.697,0.672,0.744,0.417
5,2,0.672,0.649,0.749,0.695,0.749,0.594,0.343
6,0,0.714,0.752,0.637,0.690,0.637,0.791,0.427
7,1,0.710,0.737,0.653,0.692,0.653,0.767,0.419
8,2,0.710,0.674,0.813,0.737,0.813,0.606,0.420
9,0,0.741,0.753,0.719,0.735,0.719,0.764,0.482


In [ ]:
metrics_df_CNN.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/Time Domain/CNN_GRU/Theta_time_CNN.csv', index = False)

# gru


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model_gru(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(GRU(256, return_sequences=True))
    model.add(GRU(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning_gru(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model_gru(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model_gru(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Theta/CNN_GRU/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Theta/CNN_GRU/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


In [ ]:
global_model_gru, metrics_df_gru = federated_learning_gru(Theta_data)

Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 18, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 9, 256)            0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 8s 53ms/step - loss: 1.4311 - accuracy: 0.4989 - val_loss: 1.4288 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 0s 12ms/step - loss: 1.4253 - accuracy: 0.5113 - val_loss: 1.4238 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 0s 15ms/step - loss: 1.4196 - accuracy: 0.5030 - val_loss: 1.4188 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 1s 26ms/step - loss: 1.4145 - accuracy: 0.5081 - val_loss: 1.4138 - val_accuracy: 0.4892
Epoch 5/100
29/29 [==============================] - 1s 27ms/step - loss: 1.4087 - accuracy: 0.5129 - val_loss: 1.4089 - val_accuracy: 0.6207
Epoch 6/100
29/29 [==============================] - 0s 16ms/step - loss: 1.4032 - accuracy: 0.5129 - val_loss: 1.4040 - val_accuracy: 0.5377
Epoch 7/100
29/29 [==============================] - 0s 17ms/step - loss: 1.3971 - accuracy: 0.5461 - val_loss: 1.3990 - val_accuracy: 0.5388
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/100
28/28 [==============================] - 0s 14ms/step - loss: 1.4254 - accuracy: 0.5062 - val_loss: 1.4242 - val_accuracy: 0.4955
Epoch 3/100
28/28 [==============================] - 0s 13ms/step - loss: 1.4206 - accuracy: 0.5008 - val_loss: 1.4194 - val_accuracy: 0.4955
Epoch 4/100
28/28 [==============================] - 1s 23ms/step - loss: 1.4152 - accuracy: 0.5059 - val_loss: 1.4147 - val_accuracy: 0.6007
Epoch 5/100
28/28 [==============================] - 1s 20ms/step - loss: 1.4099 - accuracy: 0.5040 - val_loss: 1.4100 - val_accuracy: 0.5158
Epoch 6/100
28/28 [==============================] - 1s 19ms/step - loss: 1.4040 - accuracy: 0.5323 - val_loss: 1.4054 - val_accuracy: 0.5045
Epoch 7/100
28/28 [==============================] - 0s 17ms/step - loss: 1.3986 - accuracy: 0.5374 - val_loss: 1.4008 - val_accuracy: 0.5045
Epoch 8/100
28/28 [==============================] - 0s 17ms/step - loss: 1.3919 - accuracy: 0.5475 - val_loss: 1.3962 - val_accuracy: 0.5045
Epoch 

...

29/29 [==============================] - 8s 51ms/step - loss: 0.4555 - accuracy: 0.8879 - val_loss: 0.8966 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 1s 21ms/step - loss: 0.4352 - accuracy: 0.8944 - val_loss: 0.8815 - val_accuracy: 0.4925
Epoch 3/100
29/29 [==============================] - 1s 22ms/step - loss: 0.4041 - accuracy: 0.9106 - val_loss: 0.8678 - val_accuracy: 0.5259
Epoch 4/100
29/29 [==============================] - 1s 21ms/step - loss: 0.4113 - accuracy: 0.9089 - val_loss: 0.8600 - val_accuracy: 0.5442
Epoch 5/100
29/29 [==============================] - 1s 21ms/step - loss: 0.3941 - accuracy: 0.9146 - val_loss: 0.8467 - val_accuracy: 0.6024
Epoch 6/100
29/29 [==============================] - 1s 22ms/step - loss: 0.4054 - accuracy: 0.9098 - val_loss: 0.8399 - val_accuracy: 0.6261
Epoch 7/100
29/29 [==============================] - 1s 21ms/step - loss: 0.4034 - accuracy: 0.9103 - val_loss: 0.8352 - val_accuracy: 0.6401
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 9s 58ms/step - loss: 0.4489 - accuracy: 0.8899 - val_loss: 0.8870 - val_accuracy: 0.4966
Epoch 2/100
28/28 [==============================] - 1s 25ms/step - loss: 0.3916 - accuracy: 0.9177 - val_loss: 0.8795 - val_accuracy: 0.5023
Epoch 3/100
28/28 [==============================] - 1s 21ms/step - loss: 0.4095 - accuracy: 0.9012 - val_loss: 0.8676 - val_accuracy: 0.5170
Epoch 4/100
28/28 [==============================] - 1s 23ms/step - loss: 0.4040 - accuracy: 0.9095 - val_loss: 0.8571 - val_accuracy: 0.5781
Epoch 5/100
28/28 [==============================] - 1s 21ms/step - loss: 0.3902 - accuracy: 0.9205 - val_loss: 0.8513 - val_accuracy: 0.5928
Epoch 6/100
28/28 [==============================] - 4s 133ms/step - loss: 0.3929 - accuracy: 0.9143 - val_loss: 0.8433 - val_accuracy: 0.6176
Epoch 7/100
28/28 [==============================] - 1s 26ms/step - loss: 0.4035 - accuracy: 0.9032 - val_loss: 0.8410 - val_accuracy: 0.6346
Epoch 8/100
28/28

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 7s 54ms/step - loss: 0.4620 - accuracy: 0.8829 - val_loss: 0.8900 - val_accuracy: 0.4855
Epoch 2/100
31/31 [==============================] - 1s 28ms/step - loss: 0.4313 - accuracy: 0.8995 - val_loss: 0.8791 - val_accuracy: 0.4959
Epoch 3/100
31/31 [==============================] - 1s 29ms/step - loss: 0.4242 - accuracy: 0.8992 - val_loss: 0.8664 - val_accuracy: 0.5186
Epoch 4/100
31/31 [==============================] - 1s 28ms/step - loss: 0.4185 - accuracy: 0.9023 - val_loss: 0.8554 - val_accuracy: 0.5640
Epoch 5/100
31/31 [==============================] - 1s 29ms/step - loss: 0.4151 - accuracy: 0.9003 - val_loss: 0.8421 - val_accuracy: 0.6250
Epoch 6/100
31/31 [==============================] - 1s 27ms/step - loss: 0.4083 - accuracy: 0.9109 - val_loss: 0.8323 - val_accuracy: 0.6539
Epoch 7/100
31/31 [==============================] - 0s 14ms/step - loss: 0.4075 - accuracy: 0.9031 - val_loss: 0.8253 - val_accuracy: 0.6467
Epoch 8/100
31/31 

In [ ]:
metrics_df_gru

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.666667,0.711253,0.561139,0.627341,0.561139,0.772194,0.333333
1,1,0.672316,0.714035,0.574859,0.636933,0.574859,0.769774,0.344633
2,2,0.647590,0.623946,0.742972,0.678277,0.742972,0.552209,0.295181
3,0,0.681742,0.725572,0.584590,0.647495,0.584590,0.778894,0.363484
4,1,0.701977,0.730645,0.639831,0.682229,0.639831,0.764124,0.403955
5,2,0.650602,0.622150,0.767068,0.687050,0.767068,0.534137,0.301205
6,0,0.723618,0.761252,0.651591,0.702166,0.651591,0.795645,0.447236
7,1,0.716808,0.717730,0.714689,0.716207,0.714689,0.718927,0.433616
8,2,0.719880,0.692443,0.791165,0.738519,0.791165,0.648594,0.439759
9,0,0.746231,0.759717,0.720268,0.739467,0.720268,0.772194,0.492462


In [ ]:
metrics_df_gru.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/Time Domain/CNN_GRU/Theta_time_gru.csv', index = False)